In [20]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:200]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)
print('Training data:', trainCIR.shape)
print('Testing data:', testCIR.shape)
# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 2 # Adjust this number based on your data
n_nonzero_coefs = 1  # Number of non-zero coefficients in sparse coding


Training data: (160, 15, 251, 2)
Testing data: (40, 15, 251, 2)


In [21]:
def reduce_data_dimension(data, n_components):
    # data: (cirs, 251, 2)
    
    reformed_cirs = []    
    for idx, cir in enumerate(data):
        products = np.abs(cir[:, 0] * cir[:, 1])
        # products = [12, 5, 0, 6, 12]

        # np.argsort(products) = [2, 1, 3, 0, 4]
        # np.argsort(products)[::-1] = [4, 0, 3, 1, 2] index in reverse order
        top_indices = np.argsort(products)[::-1][:n_components]
        top_components = cir[top_indices]
        reformed_cirs.append(top_components)
        
        # draw CIR ------------
        # # Plot the entire CIR (251 data points)
        # plt.plot(range(251), products, label='All Data Points', color='blue')
        # # Highlight the 4 most important data points
        # plt.scatter(top_indices, products[top_indices], color='red', label='Top 4 Points', s=100)
        
        # # Labeling the plot
        # plt.xlabel('Data Point Index')
        # plt.ylabel('Importance Metric (Real * Imaginary)')
        # plt.title('Comparison of All CIR Points with Top 4 Important Points')
        # plt.legend()
        # plt.show()

    reformed_cirs = np.array(reformed_cirs)
    
    return reformed_cirs


In [22]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_data_combined  = np.vstack((train_alice_cirs, train_eve_cirs)) # (320, 251, 2)

scaler = StandardScaler()
# n_nonzero_coefs <= n_components
dl = DictionaryLearning(n_components=n_components, transform_algorithm='lasso_lars', n_jobs=-1)

train_sparse_cir = []
for cir in train_data_combined:
    cir_scaled = scaler.fit_transform(cir)
    cir_sprase_transformed = dl.fit_transform(cir_scaled) # (251, 2)    
    train_sparse_cir.append(cir_sprase_transformed)

train_sparse_cir = np.array(train_sparse_cir)
print(train_sparse_cir.shape)
train_sparse_cir = reduce_data_dimension(train_sparse_cir, 4)
print(train_sparse_cir.shape)

(320, 251, 2)
(320, 4, 2)


In [23]:
atoms = np.array(train_sparse_cir)
atoms_reshape = atoms.reshape(atoms.shape[0],-1)
D = atoms_reshape.T
print('Dictionary shape:', D.shape)

Dictionary shape: (8, 320)


In [24]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
test_data_combined = np.vstack((test_alice_CIRs, test_eve_CIRs))

test_sparse_cir = []
for cir in test_data_combined:
    cir_scaled = scaler.fit_transform(cir)
    cir_transformed = dl.fit_transform(cir_scaled) # (251, 2)
    test_sparse_cir.append(cir_transformed)

test_sparse_cir = np.array(test_sparse_cir)
print('Test shape:', test_sparse_cir.shape)
test_sparse_cir = reduce_data_dimension(test_sparse_cir, 4)
print('Test reduce shape:', test_sparse_cir.shape)
test_sparse_cir_reshape = test_sparse_cir.reshape(test_sparse_cir.shape[0],-1)
print('Test reshape:', test_sparse_cir_reshape.shape)

Test shape: (80, 251, 2)
Test reduce shape: (80, 4, 2)
Test reshape: (80, 8)


In [25]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_CIRs.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_CIRs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))
print('Test labels:', test_labels.shape)

Test labels: (80,)


In [31]:
# ---------------------------------------------- Sparse Coding & Classification ------------------------------------------------
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    # print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for cir in test_sparse_cir_reshape:
    predicted_class = classify_signal(cir, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
print(predictions.shape)


(80,)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 52.50%
tp: 21
tn: 21
fp: 19
fn: 19
MDR: 0.475
FAR: 0.475
AR: 0.525
